In [11]:
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn
import statistics

from tensorflow import keras
from keras.models import Model
from keras.layers import Input

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [12]:
with open('correct_power.csv', newline='') as f:
    reader = csv.reader(f)
    power_consumption = list(reader)

power_consumption = [float(x) for x in power_consumption[0]]

[6.875, 6.943, 6.951, 6.946, 6.955, 6.956, 6.955, 6.945, 6.961, 6.899, 6.897, 6.895, 6.896, 6.9, 6.904, 6.907, 6.905, 6.912, 6.912, 6.904, 6.907, 6.959, 6.948, 6.949, 6.952, 6.96, 6.952, 6.959, 6.955, 6.954, 6.954, 6.952, 6.944, 6.957, 6.951, 6.952, 6.949, 6.954, 6.954, 6.96, 6.95, 6.957, 6.949, 6.959, 6.96, 6.955, 6.961, 6.961, 6.956, 6.955, 6.948, 6.956, 6.948, 6.957, 6.953, 6.946, 6.948, 6.951, 6.946, 6.951, 6.936, 6.948, 6.947, 6.955, 6.944, 6.955, 6.948, 6.954, 6.955, 6.959, 6.945, 6.953, 6.961, 6.95, 6.953, 6.952, 6.958, 6.954, 6.954, 6.949, 6.95, 6.952, 6.955, 6.966, 6.948, 6.953, 6.954, 6.949, 6.949, 6.96, 6.947, 6.943, 6.952, 6.957, 6.958, 6.955, 6.956, 6.952, 6.95, 6.903, 6.893, 6.891, 6.902, 6.899, 6.903, 6.902, 6.899, 6.894, 6.902, 6.9, 6.901, 6.899, 6.901, 6.9, 6.914, 6.894, 6.893, 6.901, 6.904, 6.892, 6.888, 6.901, 6.888, 6.894, 6.888, 6.903, 6.901, 6.899, 6.901, 6.896, 6.904, 6.895, 6.908, 6.897, 6.896, 6.894, 6.896, 6.902, 6.894, 6.907, 6.906, 6.906, 6.894, 6.903, 6.893

In [13]:
model_info = pd.read_csv("final_info.csv")
model_info.drop(["Unnamed: 0","Index"], inplace=True, axis=1)
model_info['PowerConsumption'] = power_consumption

In [15]:
model_info.dtypes

PredictedAccuracy    float64
RMSE                 float64
D0                     int64
D1                     int64
D2                     int64
D3                     int64
D4                     int64
ImageSize              int64
ModelSize            float64
PowerConsumption     float64
dtype: object

In [16]:
statistics.stdev(model_info["PowerConsumption"])

0.10909043084851239

In [17]:
correlation = model_info.corr()
correlation.to_csv("correlation.csv")

In [18]:
model_info = (model_info-model_info.min())/(model_info.max()-
                                                     model_info.min())

In [93]:
model_info.head()

,PredictedAccuracy,RMSE,D0,D1,D2,D3,D4,ImageSize,ModelSize,PowerConsumption
0,1.0,0.999214,0.0,0.5,1.0,1.0,1.0,1.0,0.629743,0.923035
1,1.0,0.999214,0.0,0.5,1.0,1.0,1.0,1.0,0.558225,0.960418
2,1.0,0.999214,0.0,0.5,1.0,1.0,1.0,1.0,0.693994,0.964816
3,1.0,0.999214,0.0,0.5,1.0,1.0,1.0,1.0,0.693994,0.962067
4,1.0,0.999214,0.0,0.5,1.0,1.0,1.0,1.0,0.693994,0.967015


In [94]:
# Activation function
AFs = ["logistic", "relu"]
# Hidden layer size
HLSs = [20, 50, 75, 100]
# Solver
Ss = ["lbfgs", "sgd"]
# Alpha
As = np.logspace(0.0001, 0.1, 3)
# Max iterations
MIs = [100, 200]

MLPReg = MLPRegressor(activation="logistic",hidden_layer_sizes=(100,),
                      solver="sgd",alpha=0.0001,learning_rate="constant",
                      max_iter=200,early_stopping=True)

MLPReg = GridSearchCV(estimator=MLPReg, 
                      param_grid=dict(activation=AFs,hidden_layer_sizes=HLSs,
                                      solver=Ss,alpha=As,
                                      learning_rate=["constant"],max_iter=MIs,
                                      early_stopping=[True]),n_jobs=-1)

MLPReg.fit(X=model_info.values,y=model_info["PowerConsumption"])
MLPRegCrossValScore = cross_val_score(estimator=MLPReg, X=model_info.values,
                                      y=model_info["PowerConsumption"],
                                      cv=5,n_jobs=-1)

In [95]:
print("\nCross validation scores: " + str(MLPRegCrossValScore))
print("Average score: " + str(statistics.mean(MLPRegCrossValScore)))


Cross validation scores: [0.95268754 0.97389123 0.97969516 0.9911869  0.98943847]
Average score: 0.977379859335638


In [96]:
MLPReg.best_params_

{'activation': 'relu',
 'alpha': 1.0002302850208247,
 'early_stopping': True,
 'hidden_layer_sizes': 50,
 'learning_rate': 'constant',
 'max_iter': 200,
 'solver': 'lbfgs'}